In [ ]:
import time, json
from datetime import datetime, timezone

def utc_now():
    return datetime.now(timezone.utc).isoformat()

try: runId = runId
except NameError: runId = f"manual_{int(time.time())}"

try: workloadName = workloadName
except NameError: workloadName = "unknown"

try: tier = tier
except NameError: tier = "unknown"

try: weight = weight
except NameError: weight = "0"


In [ ]:
import time, json
from pyspark.sql import functions as F

start_ts = time.time()
start_utc = utc_now()

# Small dataset (~1M)
n = 1_000_000
df = (spark.range(0, n)
          .withColumn("pickup_minute", (F.col("id") % 1440).cast("int"))
          .withColumn("fare_amount", (F.rand(seed=42) * 50).cast("double")))

# Light aggregation (tiny shuffle)
agg = (df.groupBy("pickup_minute")
         .agg(F.count("*").alias("trip_count"),
              F.avg("fare_amount").alias("avg_fare")))

rows = agg.count()  # force execution

duration = round(time.time() - start_ts, 2)
result = {
    "runId": runId, "workloadName": workloadName, "tier": tier, "weight": weight,
    "startTimeUtc": start_utc, "durationSec": duration,
    "metric": {"resultRows": rows}
}
print(result)
mssparkutils.notebook.exit(json.dumps(result))


In [ ]:
mssparkutils.notebook.exit(json.dumps(result))
